# Data Engineering 1: Graded Lab 01
---------------

#### Grading
For this graded lab you can get a total of 10 points. These 10 points count 10% of your final grade for the course.

#### Start
Start of the Graded Lab 01 is **Thursday, March 21st at 23:55**.

#### Deadline
Deadline for the submission of the Graded Lab 01 is **Thursday, April 11th at 23:59**.

#### Note
Check each result carefully. Use data filter, cleaning, and transformation methods wherever needed. The data can sometimes be really messy and have hidden issues.

#### Submission
You are allowed to submit the solution in groups of **two or three** students.
Submit your GradedLab01.ipynb file renamed to FirstnameStudent01LastnameStudent01_FirstnameStudent02LastnameStudent02_FirstnameStudent03LastnameStudent03.ipynb in moodle.   
Please submit a runnable python jupyter notebook file.
All other submissions will be rejected and graded with 0 points.

##### Task 01: Structured Data [4 points].    
The imdb.csv file contains a dataset extracted from IMDB with several attributes. For example, the title, plot, and the language of the movies. Read the imdb.csv file in a pandas dataframe and try to answer the following questions.

##### __(a)  What is the min, max, and the average value of the attribute 'rating' for all movies? [0.5 points].__ 

In [ ]:
import datetime

import numpy as np
!pip install pandas

In [160]:
import pandas as pd

data = pd.read_csv('imdb.csv')

data.head()

print(f"Min: {data.rating.min()}")
print(f"Max: {data.rating.max()}")
print(f"Average: {data.rating.mean():.1f}")

Min: 8.1
Max: 9.6
Average: 8.6


##### __(b) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' for all movies? [0.5 points].__ 

In [161]:
data["cumulative worldwide gross"] = data["cumulative worldwide gross"].str.replace(r'\s.+', '', regex=True).replace(r'\D', '', regex=True).astype(float)

print(f"Min: {data['cumulative worldwide gross'].min()}")
print(f"Max: {data['cumulative worldwide gross'].max()}")
print(f"Average: {data['cumulative worldwide gross'].mean():.1f}")

Min: 0.0
Max: 2797800564.0
Average: 238223230.2


##### __(c) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' grouped by genre of the movies? [1 point].__ 

In [162]:
data['genres'] = data['genres'].apply(lambda x: x.replace("'", '').strip('][').split(', ') if not isinstance(x, list) else x)
data = data.explode('genres')
data.groupby(by=['genres'], dropna=True)['cumulative worldwide gross'].min()
data.groupby(by=['genres'], dropna=True)['cumulative worldwide gross'].max()
data.groupby(by=['genres'], dropna=True)['cumulative worldwide gross'].mean()

genres
Action         4.655257e+08
Adventure      5.123588e+08
Animation      3.958353e+08
Biography      1.538608e+08
Comedy         2.447827e+08
Crime          1.824480e+08
Documentary    0.000000e+00
Drama          2.290571e+08
Family         5.497713e+08
Fantasy        3.426368e+08
Game-Show      0.000000e+00
History        7.840920e+07
Horror         8.982547e+07
Music          2.946962e+08
Musical        6.325005e+08
Mystery        1.406539e+08
News           0.000000e+00
Reality-TV     0.000000e+00
Romance        2.344835e+08
Sci-Fi         4.410689e+08
Sport          1.164948e+08
Talk-Show      0.000000e+00
Thriller       2.154442e+08
War            1.213808e+08
Western        3.362419e+08
Name: cumulative worldwide gross, dtype: float64

##### __(d) What is the profit (defined as cumulative worldwide gross - budget) of each movie? [1 point].__ 

In [163]:
data.budget = data.budget.str.replace(r'\D', '', regex=True).astype(float)
profit = data['cumulative worldwide gross'] - data.budget
profit

0       33500000.0
1      239066411.0
1      239066411.0
2      819558444.0
2      819558444.0
          ...     
156            0.0
156            0.0
157            0.0
157            0.0
158            0.0
Length: 487, dtype: float64

##### __(e)  What is the min, max, and the average value of the atttribute 'opening weekend united states' per month? [1 point].__ 

In [172]:
import datetime

data['opening weekend united states']= data['opening weekend united states'].apply(lambda x: x.replace("$", '').split(', ') if not isinstance(x, list) else x)
data = data.merge(data['opening weekend united states'].apply(lambda s: pd.Series({'opening weekend united states income': float(s[0].replace(',', '')), 'opening weekend united states dates': datetime.datetime.strptime(s[1], '%d %b %Y')}) if len(s) > 1 else pd.Series({'opening weekend united states income': np.NaN, 'opening weekend united states dates': np.NaN})), left_index=True, right_index=True)

data.groupby(by=[data['opening weekend united states dates'].dt.month], dropna=True)['opening weekend united states income'].min()
data.groupby(by=[data['opening weekend united states dates'].dt.month], dropna=True)['opening weekend united states income'].max()
data.groupby(by=[data['opening weekend united states dates'].dt.month], dropna=True)['opening weekend united states income'].mean()

opening weekend united states dates
1.0     1.326144e+07
2.0     2.974682e+07
3.0     3.421716e+07
4.0     1.593046e+08
5.0     5.929679e+07
6.0     5.916076e+07
7.0     5.430706e+07
8.0     1.121913e+07
9.0     1.056989e+07
10.0    2.193238e+07
11.0    2.672944e+07
12.0    1.964329e+07
Name: opening weekend united states income, dtype: float64

#### Task 02: Unstructured Data [6 points]. 
This time use the files imdb.csv from Task 01 and the imdb2.csv file. In this task we mainly use the content of the attribute 'plot' for the implementation of a retrieval system.

##### __(a)  Read the files imdb.csv and imdb2.csv in a dataframe called imdb. Add a column which is called 'plotterms' to each item. The new column should contain all terms (lower-case and cleaned from special characters) from the plot attribute which are not contained in the stopwords_english.txt file. [0.5 points]__ 

In [204]:
!spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 4.1 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 6.8 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 9.7 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 12.9 MB/s eta 0:00:01
     ---------------- ----------------------- 5.3/12.8 MB 16.8 MB/s eta 0:00:01
     ------------------------ --------------- 7.8/12.8 MB 21.7 MB/s eta 0:00:01
     ---------------------------------- ---- 11.4/12.8 MB 43.5 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 50.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [211]:
import pandas as pd
import spacy
import re

nlp = spacy.load('en_core_web_sm')

data = pd.read_csv('imdb2.csv')
imdb = pd.read_csv('imdb.csv')

imdb = pd.concat([data, imdb])

with open('stopwords_english.txt', 'r+', encoding='utf-8') as file:
    stopwords = file.readlines()

def tokenize(text):
    doc = nlp(text)
    return [token.text.lower() for token in doc if token.text not in stopwords]

imdb['plot'] = imdb['plot'].apply(lambda x: '. '.join([re.sub(r'[^\w\s]', '', y) for y in x.split(', ')]))
imdb['plotterms'] = imdb['plot'].apply(tokenize)
imdb

,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms
0,120669,movie,Fear and Loathing in Las Vegas,['English'],"['Adventure', 'Comedy', 'Drama']",7.5,Terry Gilliam,0,0,0,An oddball journalist and his psychopathic law...,"[an, oddball, journalist, and, his, psychopath..."
1,1343092,movie,The Great Gatsby,['English'],"['Drama', 'Romance']",7.2,Baz Luhrmann,0,0,0,A writer and wall street trader. Nick. finds h...,"[a, writer, and, wall, street, trader, ., nick..."
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,Dave has the American Dream and hates it His e...,"[dave, has, the, american, dream, and, hates, ..."
3,2267368,movie,Joshua Tree,['English'],['Drama'],6.6,Li Cheng,0,0,0,Set in the US heartland following the 2008 eco...,"[set, in, the, us, heartland, following, the, ..."
4,13560124,movie,After America,"['English', 'Somali']","['Drama', 'Fantasy', 'Mystery']",4.5,Jake Yuzna,0,0,0,A group of criminal justice deescalation worke...,"[a, group, of, criminal, justice, deescalation..."
...,...,...,...,...,...,...,...,...,...,...,...,...
154,2085059,tv series,Black Mirror,['English'],"['Drama', 'Mystery', 'Sci-Fi', 'Thriller']",8.8,NaN,000000000,000000000,000000000,An anthology series exploring a twisted. hight...,"[an, anthology, series, exploring, a, twisted,..."
155,2707408,tv series,Narcos,"['English', 'Spanish']","['Biography', 'Crime', 'Drama', 'Thriller']",8.8,NaN,000000000,000000000,000000000,A chronicled look at the criminal exploits of ...,"[a, chronicled, look, at, the, criminal, explo..."
156,98936,tv series,Twin Peaks,"['English', 'Icelandic', 'Afrikaans', 'Norwegi...","['Crime', 'Drama', 'Mystery', 'Thriller']",8.8,NaN,000000000,000000000,000000000,An idiosyncratic FBI agent investigates the mu...,"[an, idiosyncratic, fbi, agent, investigates, ..."
157,4934214,tv series,Taskmaster,['English'],"['Comedy', 'Game-Show']",9.0,NaN,000000000,000000000,000000000,Five comedians are set tasks challenging their...,"[five, comedians, are, set, tasks, challenging..."


##### __(b)  Create an inverted index for the terms in the 'plotterms' column. Use a datastructure of your choice for the implementation [0.5 points]__ 

In [212]:
index = {}
count = 0

def append_index(plotterms):
    for term in plotterms:
        if index.get(term) is None:
            index[term] = len(index)
imdb['plotterms'].apply(append_index)
index

{'an': 0,
 'oddball': 1,
 'journalist': 2,
 'and': 3,
 'his': 4,
 'psychopathic': 5,
 'lawyer': 6,
 'travel': 7,
 'to': 8,
 'las': 9,
 'vegas': 10,
 'for': 11,
 'a': 12,
 'series': 13,
 'of': 14,
 'psychedelic': 15,
 'escapades': 16,
 '.': 17,
 'the': 18,
 'bigscreen': 19,
 'version': 20,
 'hunter': 21,
 's': 22,
 'thompsons': 23,
 'seminal': 24,
 'classic': 25,
 'about': 26,
 'road': 27,
 'trip': 28,
 'across': 29,
 'western': 30,
 'america': 31,
 'as': 32,
 'he': 33,
 'large': 34,
 'samoan': 35,
 'searched': 36,
 'desperately': 37,
 'american': 38,
 'dream': 39,
 'they': 40,
 'were': 41,
 'helped': 42,
 'in': 43,
 'part': 44,
 'by': 45,
 'huge': 46,
 'amount': 47,
 'drugs': 48,
 'alcohol': 49,
 'kept': 50,
 'their': 51,
 'convertible': 52,
 'red': 53,
 'sharklaurence': 54,
 'mixson': 55,
 'raoul': 56,
 'duke': 57,
 'is': 58,
 'drug': 59,
 'addled': 60,
 'gonzo': 61,
 'sent': 62,
 'cover': 63,
 'motorcycle': 64,
 'race': 65,
 'article': 66,
 'magazine': 67,
 'but': 68,
 'then': 69,
 '

##### __(c) Take the inverted index and create the posting list for the query 'american'. Return the posting list as well as the top5 items scored by the rating attribute. [0.5 points]__ 

##### __(d) Take the inverted index and create the posting lists for the query terms 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot). Your merging algorithm should be efficient and reduce the number of comparison to a minimum. Don't use existing python methods (like intersect or in) for the merge algorithm. Return the merged posting list as well as the top5 items scored by the rating attribute. [1 point]__ 

##### __(e) Extend your index to be able to rank the resulting items from Task 2d by the occurrence of search terms in the plotterms. Execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) again and print the resulting items.  [1 point]__

##### __(f) Use the TF-IDF score and the cosine similarity to execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) against all plots in the collection. Print the results with the corresponding ranking score. Also execute the query 'american' or 'dream' (the term 'american' or the term 'dream' should be contained in the plot).  [1.5 points]__